<a href="https://colab.research.google.com/github/sahithikodali1/Summarization-of-Biomedical-evidence/blob/Data_processing/BIOASQ_Baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import os

print(os.listdir("/content/drive/MyDrive/Thesis_B"))

['rouge_8b_mod.csv', '.ipynb_checkpoints', 'test_data.csv', 'train_data.csv']


In [ ]:
DATA_DIR = "/content/drive/MyDrive/Thesis_B"

train_data = pd.read_csv(os.path.join(DATA_DIR,'train_data.csv'))
# print(train_data.head()) 

print(train_data.columns.values)
test_data = pd.read_csv(os.path.join(DATA_DIR,'test_data.csv'))
# print(test_data[:10]) 

['qid' 'pubmedid' 'sentid' 'N1' 'N2' 'L' 'S4' 'SU4' 'sentence text'
 'SU4_labels']


In [ ]:
# install
!pip install pytorch-pretrained-bert pytorch-nlp

# BERT imports
import torch

from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from pytorch_pretrained_bert import BertTokenizer, BertConfig
from pytorch_pretrained_bert import BertAdam, BertForSequenceClassification
from tqdm import tqdm, trange

import pandas as pd
import io
import numpy as np
import matplotlib.pyplot as plt
% matplotlib inline

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

In [ ]:
def data_grouplists(data):
  data_list = train_data.groupby('qid', as_index=False)['SU4_labels','sentence text'].agg(list)
  return data_list

In [ ]:
train_list = data_grouplists(train_data)
test_list = data_grouplists(test_data)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  


In [ ]:
def add_specialtokens(data_list):
  specialtok_sentences = []

  for sentence in data_list['sentence text']:
    each_specialtoken = []
    for each in sentence:
      each_specialtoken.append(["[CLS] " + each + " [SEP]" ])
    specialtok_sentences.append(each_specialtoken)
  return specialtok_sentences

In [ ]:
train_specialtok = add_specialtokens(train_list)
test_specialtok = add_specialtokens(test_list)

In [ ]:
print(train_specialtok[0])
print(train_specialtok[1])

[["[CLS] The non-Mendelian inheritance of sporadic non-syndromic Hirschsprung's disease proved to be complex; involvement of multiple loci was demonstrated in a multiplicative model [SEP]"], ["[CLS] The non-Mendelian inheritance of sporadic non-syndromic Hirschsprung's disease proved to be complex; involvement of multiple loci was demonstrated in a multiplicative model [SEP]"], ["[CLS] The majority of the identified genes are related to Mendelian syndromic forms of Hirschsprung's disease [SEP]"], ["[CLS] The majority of the identified genes are related to Mendelian syndromic forms of Hirschsprung's disease. [SEP]"], ["[CLS] In this study, we review the identification of genes and loci involved in the non-syndromic common form and syndromic Mendelian forms of Hirschsprung's disease. [SEP]"], ['[CLS] In the etiology of Hirschsprung disease various genes play a role; these are: RET, EDNRB, GDNF, EDN3 and SOX10, NTN3, ECE1, Mutations in these genes may result in dominant, recessive or mult

In [ ]:
def tokenize_sentences(specialtok_sentences):
  tokenized_sentences = []

  for sentence in specialtok_sentences:
    eachsent_tokenized = []
    for each in sentence:
      eachsent_tokenized.append(tokenizer.tokenize(each[0]))
    tokenized_sentences.append(eachsent_tokenized)
  return tokenized_sentences

In [ ]:
train_tokenized = tokenize_sentences(train_specialtok)
test_tokenized = tokenize_sentences(test_specialtok)


In [ ]:
def data_tokenizedsent(tokenized_sentences, data_list):
  data_list['tokenized_sentences'] = tokenized_sentences
  return data_list

In [ ]:
print(train_tokenized[0])

[['[CLS]', 'the', 'non', '-', 'men', '##del', '##ian', 'inheritance', 'of', 'sporadic', 'non', '-', 'syn', '##dro', '##mic', 'hirsch', '##sp', '##run', '##g', "'", 's', 'disease', 'proved', 'to', 'be', 'complex', ';', 'involvement', 'of', 'multiple', 'lo', '##ci', 'was', 'demonstrated', 'in', 'a', 'multi', '##pl', '##icative', 'model', '[SEP]'], ['[CLS]', 'the', 'non', '-', 'men', '##del', '##ian', 'inheritance', 'of', 'sporadic', 'non', '-', 'syn', '##dro', '##mic', 'hirsch', '##sp', '##run', '##g', "'", 's', 'disease', 'proved', 'to', 'be', 'complex', ';', 'involvement', 'of', 'multiple', 'lo', '##ci', 'was', 'demonstrated', 'in', 'a', 'multi', '##pl', '##icative', 'model', '[SEP]'], ['[CLS]', 'the', 'majority', 'of', 'the', 'identified', 'genes', 'are', 'related', 'to', 'men', '##del', '##ian', 'syn', '##dro', '##mic', 'forms', 'of', 'hirsch', '##sp', '##run', '##g', "'", 's', 'disease', '[SEP]'], ['[CLS]', 'the', 'majority', 'of', 'the', 'identified', 'genes', 'are', 'related', 'to

In [ ]:
data_train_tokenized = data_tokenizedsent(train_tokenized, train_list)
data_test_tokenized = data_tokenizedsent(test_tokenized, test_list)

In [ ]:
print(len(train_tokenized))
print(data_train_tokenized.columns.values)

2594
['qid' 'SU4_labels' 'sentence text' 'tokenized_sentences']


In [ ]:
print(len(train_tokenized[0]))
print(len(train_tokenized[1]))

21
16


In [ ]:
def token2ids(tokenized_sentences):
  # Set the maximum sequence length. 
  MAX_LEN = 512
  token_ids = []

  for tokenized_sentence in tokenized_sentences:
    tokens_to_ids = [tokenizer.convert_tokens_to_ids(each) for each in tokenized_sentence]
    tokens_to_ids = pad_sequences(tokens_to_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
    token_ids.append(tokens_to_ids)
  return token_ids

In [ ]:
train_tokens2ids = token2ids(train_tokenized)
test_tokens2ids = token2ids(test_tokenized)

Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (529 > 512). Running this sequence through BERT will result in indexing errors
Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (529 > 512). Running this sequence through BERT will result in indexing errors


In [ ]:
print(len(train_tokens2ids))
print(len(train_tokens2ids[0][20]))
print(len(train_tokens2ids[1]))

2594
512
16


In [ ]:
def create_masks(token_ids):
  attention_masks = []

  for tid in token_ids:
    eachtid_mask = []
    for each in tid:
      mask = [float(i>0) for i in each] #for each token 
      eachtid_mask.append(mask)#for each sentence
    attention_masks.append(eachtid_mask)
  return attention_masks

In [ ]:
train_masks = create_masks(train_tokens2ids)
test_masks = create_masks(test_tokens2ids)

In [ ]:
print(len(train_masks))
print(len(train_masks[0]))
print(len(train_masks[0][0]))
print(len(train_masks[1]))
print(len(train_masks[1][1]))


2594
21
512
16
512
